In [1]:
# Dependencies
import pandas as pd
import sqlalchemy
import numpy as np

In [2]:
# Import Spotify Data
spotify_data = "spotifyWeeklyTop200Streams_cleaned12.6.csv"

spotify_df = pd.read_csv(spotify_data)
spotify_df.head(50)

,Unnamed: 0,Name,Artist,Features,Streams,Week
0,0,In My Feelings,Drake,NaN,30747676,7/20/2018
1,1,Lucid Dreams,Juice WRLD,NaN,12930705,7/20/2018
2,2,Nonstop,Drake,NaN,12312859,7/20/2018
3,3,God is a woman,Ariana Grande,NaN,10771324,7/20/2018
4,4,SAD!,XXXTENTACION,NaN,10503061,7/20/2018
5,5,Taste,Tyga,Offset,9777479,7/20/2018
6,6,Don't Matter To Me,Drake,NaN,9769263,7/20/2018
7,7,I Like It,Cardi B,NaN,9278476,7/20/2018
8,8,Moonlight,XXXTENTACION,NaN,8544143,7/20/2018
9,9,Girls Like You,Maroon 5,Cardi B,8257586,7/20/2018


In [3]:
# Drop Features Column in Spotify DataFrame
spotify_df = spotify_df.drop(columns = ['Features', 'Unnamed: 0'])
spotify_df.head()

,Name,Artist,Streams,Week
0,In My Feelings,Drake,30747676,7/20/2018
1,Lucid Dreams,Juice WRLD,12930705,7/20/2018
2,Nonstop,Drake,12312859,7/20/2018
3,God is a woman,Ariana Grande,10771324,7/20/2018
4,SAD!,XXXTENTACION,10503061,7/20/2018


In [4]:
# Adjust the Spotify DataFrame so the start of each week lines up with Billboard
from datetime import timedelta
spotify_df["Week"] = pd.to_datetime(spotify_df["Week"])
spotify_df["one_day_behind"] = spotify_df["Week"].apply(lambda x: x - timedelta(days=1))
spotify_df["one_day_behind"] = spotify_df["one_day_behind"].dt.strftime("%m/%d/%y")
spotify_df = spotify_df.drop(columns = ["Week"])
spotify_df.head()

,Name,Artist,Streams,one_day_behind
0,In My Feelings,Drake,30747676,07/19/18
1,Lucid Dreams,Juice WRLD,12930705,07/19/18
2,Nonstop,Drake,12312859,07/19/18
3,God is a woman,Ariana Grande,10771324,07/19/18
4,SAD!,XXXTENTACION,10503061,07/19/18


In [5]:
# Rename "one_day_"
spotify_df.rename(columns = {'Name': 'Song Name',
                             'Streams': 'Weekly Streams',
                             'one_day_behind': 'Week of'}, inplace = True)
spotify_df.head()

,Song Name,Artist,Weekly Streams,Week of
0,In My Feelings,Drake,30747676,07/19/18
1,Lucid Dreams,Juice WRLD,12930705,07/19/18
2,Nonstop,Drake,12312859,07/19/18
3,God is a woman,Ariana Grande,10771324,07/19/18
4,SAD!,XXXTENTACION,10503061,07/19/18


In [6]:
# Import the data from Billboard
billboard_data = "billboardHot100_1999-2019_cleaned_12.6.csv"

billboard_df = pd.read_csv(billboard_data)
billboard_df.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,17,NC,41,NaN,NaN,08/18/18,8/3/2018,"Trap,Rap","Ryan meyer, Rich meyer, Johnny stevens, Allen ...","NC-17 \nOoh\nMe and my bitch, I swear we like ...",NaN
1,800,1,45,3.0,42.0,02/24/18,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
2,800,1,40,3.0,41.0,02/17/18,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
3,800,1,33,3.0,40.0,02/10/18,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
4,800,1,46,3.0,39.0,02/03/18,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN


In [7]:
# Clean the Billboard DataFrame
billboard_df = billboard_df.drop(columns = ['Peak.position', 
                                            'Weeks.on.chart', 
                                            'Date', 
                                            'Writing.Credits', 
                                            'Lyrics', 
                                            'Features'])
billboard_df.head()

,Artists,Name,Weekly.rank,Week,Genre
0,17,NC,41,08/18/18,"Trap,Rap"
1,800,1,45,02/24/18,"Hip-Hop,Canada,DMV,Pop,Rap"
2,800,1,40,02/17/18,"Hip-Hop,Canada,DMV,Pop,Rap"
3,800,1,33,02/10/18,"Hip-Hop,Canada,DMV,Pop,Rap"
4,800,1,46,02/03/18,"Hip-Hop,Canada,DMV,Pop,Rap"


In [8]:
# Rename columns in Clean Billboard DataFrame
billboard_df.rename(columns = {'Artists': 'Artist',
                                'Name': 'Song Name', 
                               'Weekly.rank': 'Weekly Rank', 
                               'Week': 'Week of'}, inplace = True )
billboard_df.head()

,Artist,Song Name,Weekly Rank,Week of,Genre
0,17,NC,41,08/18/18,"Trap,Rap"
1,800,1,45,02/24/18,"Hip-Hop,Canada,DMV,Pop,Rap"
2,800,1,40,02/17/18,"Hip-Hop,Canada,DMV,Pop,Rap"
3,800,1,33,02/10/18,"Hip-Hop,Canada,DMV,Pop,Rap"
4,800,1,46,02/03/18,"Hip-Hop,Canada,DMV,Pop,Rap"


In [9]:
spotify_df.head()

,Song Name,Artist,Weekly Streams,Week of
0,In My Feelings,Drake,30747676,07/19/18
1,Lucid Dreams,Juice WRLD,12930705,07/19/18
2,Nonstop,Drake,12312859,07/19/18
3,God is a woman,Ariana Grande,10771324,07/19/18
4,SAD!,XXXTENTACION,10503061,07/19/18


In [10]:
billboard_df['Artist'] = billboard_df['Artist'].str.rstrip()
billboard_df['Song Name'] = billboard_df['Song Name'].str.rstrip()

<b> Merging and Billboard and Spotify DataFrames</b>


In [23]:
df_merged = billboard_df.merge(spotify_df, on = ["Artist"], how = 'left')
df_merged

,Artist_x,Song Name,Weekly Rank,Week of,Genre,Artist_y,Weekly Streams
0,17,NC,41,08/18/18,"Trap,Rap",NaN,NaN
1,800,1,45,02/24/18,"Hip-Hop,Canada,DMV,Pop,Rap",NaN,NaN
2,800,1,40,02/17/18,"Hip-Hop,Canada,DMV,Pop,Rap",NaN,NaN
3,800,1,33,02/10/18,"Hip-Hop,Canada,DMV,Pop,Rap",NaN,NaN
4,800,1,46,02/03/18,"Hip-Hop,Canada,DMV,Pop,Rap",NaN,NaN
...,...,...,...,...,...,...,...
13277,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,"Deutschland,Electro-Pop,Electronic,Dance,Pop",NaN,NaN
13278,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,"Deutschland,Electro-Pop,Electronic,Dance,Pop",NaN,NaN
13279,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,"Deutschland,Electro-Pop,Electronic,Dance,Pop",NaN,NaN
13280,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,"Deutschland,Electro-Pop,Electronic,Dance,Pop",NaN,NaN


In [41]:
# SQL
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [42]:
connection_string = "postgres:postgres@localhost:5432/music_db"
engine = create_engine(f'postgresql://{connection_string}')

In [31]:
engine.table_names()

['billboard6']

In [43]:
engine.execute("SELECT * FROM billboard6")

In [46]:
# session = Session(bind=engine)
# songs = session.query(billboard6)
# for song in songs:
#     print(song.song_names)